# Cross sections
Plot our elastic and inelastic scattering cross-sections for hydrogen/tritium. Look angular distributions of outgoing particles.

In [1]:
# Import common libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as const

In [2]:
# Make font size bigger and allow LaTeX rendering
plt.rcParams.update({'font.size': 15})
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

The ionisation cross-sections are from M. E. Rudd, *Phys. Rev. A* **44** (1991).

In [3]:
# Import cross-section models
from src.Rudd1991 import RuddXSec
from src.Mott import MottXSec
import src.Constants as const
from src.Stone2002 import ExcitationXSec

## Total cross-sections
See what our total cross-sections look like across the tritium $\beta$-decay spectrum.

In [ ]:
incidentEArray = np.linspace(1e3, 19e3, 300)  # Incident electron energies in eV
mottXSecArray = np.zeros_like(incidentEArray)
ionisationXSecArray = np.zeros_like(incidentEArray)
for iE, E in enumerate(incidentEArray):
    mottXSecArray[iE] = MottXSec(E).TotalXSec()
    ionisationXSecArray[iE] = RuddXSec(E).TotalXSec()

# Plot the cross-sections
plt.figure()
plt.plot(incidentEArray / 1e3, mottXSecArray * 1e4, label='Mott')
plt.plot(incidentEArray / 1e3, ionisationXSecArray * 1e4, label='Ionisation')
plt.yscale('log')
plt.xlabel('Incident electron energy [keV]')
plt.ylabel('Cross-section [cm$^2$]')
plt.legend()
plt.show()

## Angular distributions for outgoing electrons

In [ ]:
endpointEnergy = 18.575e3 # eV
endpointElasticXSec = MottXSec(endpointEnergy)
endpointIonisationXSec = RuddXSec(endpointEnergy)

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
angles = np.linspace(1e-5, np.pi - 1e-4, 600)
plt.plot(angles, endpointElasticXSec.SinglyDifferentialXSec_theta(angles) * 1e4)
plt.xlabel('Scattering angle [radians]')
plt.yscale('log')
plt.ylabel('$d \sigma / d \\rm{cos}\\theta$ [cm$^2$]')
plt.title(f'Elastic scattering, T = {endpointEnergy / 1e3:.3f} keV')
plt.xlim([-0.01, np.pi])

plt.subplot(1, 2, 2)
plt.plot(angles, endpointIonisationXSec.DoublyDifferentialXSec(endpointEnergy - const.IONIZATIONENERGYH - 1, angles) * 1e4, label='1 eV')
plt.plot(angles, endpointIonisationXSec.DoublyDifferentialXSec(endpointEnergy - const.IONIZATIONENERGYH - 5, angles) * 1e4, label='5 eV')
plt.plot(angles, endpointIonisationXSec.DoublyDifferentialXSec(endpointEnergy - const.IONIZATIONENERGYH - 20, angles) * 1e4, label='20 eV')
plt.plot(angles, endpointIonisationXSec.DoublyDifferentialXSec(endpointEnergy - const.IONIZATIONENERGYH - 50, angles) * 1e4, label='50 eV')
plt.yscale('log')
plt.xlabel('Scattering angle [radians]')
plt.ylabel('$d^2 \sigma / (d \\rm{cos}\\theta dW)$ [cm$^2$ eV$^{-1}$]')
plt.xlim([0, np.pi / 4])
plt.legend(title = 'Secondary energy')
plt.title(f'Ionisation, T = {endpointEnergy / 1e3:.3f} keV')
plt.show()

## Excitation cross-sections

In [ ]:
# Plot the excitation cross-section as function of energy
exciteXSec = ExcitationXSec(100.0)
eArray2p, xSecArr2p = exciteXSec.XSec2pData()
eArray3p, xSecArr3p = exciteXSec.XSec3pData()
eArray4p, xSecArr4p = exciteXSec.XSec4pData()
eArray5p, xSecArr5p = exciteXSec.XSec5pData()
eArray6p, xSecArr6p = exciteXSec.XSec6pData()

totalXSecArr = np.array([ExcitationXSec(energy).TotalXSec() for energy in eArray2p])

plt.figure()
plt.plot(eArray2p, xSecArr2p, '-', label='1s-2p')
plt.plot(eArray3p, xSecArr3p, '-', label='1s-3p')
plt.plot(eArray4p, xSecArr4p, '-', label='1s-4p')
plt.plot(eArray5p, xSecArr5p, '-', label='1s-5p')
plt.plot(eArray6p, xSecArr6p, '-', label='1s-6p')
plt.plot(eArray2p, totalXSecArr, '--', label='Total')
plt.xscale('log')
plt.ylabel('Cross-section [m$^2$]')
plt.xlabel('Energy [eV]')
plt.title(r'$\sigma_{BE}$ for H')
plt.legend()
plt.ylim([0., 1.1 * np.max(totalXSecArr)])
plt.xlim([10., 3e3])
plt.show()

## Comparison of excitation and ionisation cross-sections

In [ ]:
# Compare excitation and ionisation cross-sections
energyArray = np.logspace(np.log10(14.0), np.log10(3e3), 300)
exciteXSecArray = np.array([ExcitationXSec(E).TotalXSec() for E in energyArray])
ionisationXSecArray = np.array([RuddXSec(E).TotalXSec() for E in energyArray])

plt.figure()
plt.plot(energyArray, exciteXSecArray, label='Excitation')
plt.plot(energyArray, ionisationXSecArray, label='Ionisation')
plt.xscale('log')
plt.ylabel('Cross-section [m$^2$]')
plt.xlabel('Energy [eV]')
plt.legend()
plt.xlim([10., 3e3])
plt.ylim([0., 1.1 * np.max(exciteXSecArray)])
plt.title('Cross-sections for atomic H')
plt.show()